In [15]:
import json
dictionary={}
hateful_or_not={}
count=0
with open ("../data/train.jsonl",'r') as f:
    for line in f:
        elem=json.loads(line)
        if (int(elem['label'])==0):
            hateful_or_not[int(elem['id'])]=0
        else:
            hateful_or_not[int(elem['id'])]=1
            count+=1
# print(count)
probab_hateful=count/len(hateful_or_not)
words_captions_hateful={}
words_captions_normal={}
with open("../data/train.jsonl", 'r') as f:
    # dictionary=json.load(f)
    for line in f:
        elem=json.loads(line)
        for word in elem['text'].split(" "):
            if (hateful_or_not[int(elem['id'])]==0):
                if word in words_captions_normal:
                    words_captions_normal[word]+=1
                else:
                    words_captions_normal[word]=1
            else:
                if word in words_captions_hateful:
                    words_captions_hateful[word]+=1
                else:
                    words_captions_hateful[word]=1


# print(words_captions_hateful)
# print(words_captions_normal)
with open("words_captions_hateful.json", 'w') as f:
    json.dump(words_captions_hateful,f)
with open("words_captions_normal.json", 'w') as f:
    json.dump(words_captions_normal,f)

{'jew': 27, 'mad?': 1, 'get': 128, 'fuhrerious!': 1, 'brother...': 1, 'a': 1064, 'day': 37, 'without': 15, 'blast': 6, 'is': 477, 'wasted': 2, 'bribing': 1, 'muslims': 123, 'for': 305, 'liberal': 11, 'votes': 1, 'justin': 4, "trudeau's": 1, 'only': 75, 'skill?': 1, 'why': 85, 'does': 27, 'trudeau': 2, 'love': 32, 'foreigners': 4, 'so': 173, 'much': 21, 'while': 22, 'openly': 1, 'disrespecting': 1, 'canadians,': 1, 'canadian': 1, 'values,': 1, 'our': 60, 'history': 13, 'and': 630, 'traditions,': 3, 'seniors': 1, 'veterans?': 1, 'sharia': 12, 'law': 9, 'has': 56, 'no': 114, 'place': 9, 'in': 464, 'canada!': 2, 'never': 45, 'has...': 1, 'will.': 2, 'd.j.': 1, 'osama': 1, 'spin': 1, 'laden': 1, "droppin'": 1, 'beats': 3, 'like': 224, 'the': 1270, 'twin': 4, 'towers': 1, 'we': 165, 'said': 27, 'would': 53, 'forget': 13, 'are': 310, 'you': 919, 'voting': 4, 'them': 87, 'into': 55, 'government?': 4, "i'm": 141, 'sick': 3, 'of': 487, 'politics': 2, "here's": 6, 'pic': 3, 'michelle': 8, 'obama'

In [59]:
probab_hateful=1/2
import numpy as np

class NaiveBayesClassifier:
    def __init__(self):
        self.class_prior = {}
        self.word_likelihoods = {}

    def train(self, word_counts, hateful_dict, normal_dict):
        # for val in hatef
        total_hateful=0
        total_normal=0
        for val in hateful_dict.values():
            total_hateful+=val
        for val in normal_dict.values():
            total_normal+=val
        # total_hateful = sum(hateful_counts)
        # total_normal = sum(normal_counts)
        total_examples = total_hateful + total_normal
        
        # self.class_prior[1] = total_hateful / total_examples
        # self.class_prior[0] = total_normal / total_examples
        self.class_prior[1] = probab_hateful
        self.class_prior[0] = 1 - probab_hateful
        
        for word in word_counts:
            hateful_count=0
            normal_count=0
            if (word in hateful_dict):
                hateful_count=int(hateful_dict[word])
            if (word in normal_dict):
                normal_count=int(normal_dict[word])
            prob_hateful = (hateful_count + 1) / (total_hateful + len(word_counts))
            prob_normal = (normal_count + 1) / (total_normal + len(word_counts))
            self.word_likelihoods[word] = (prob_hateful, prob_normal)

    def predict(self, X_test):
        predictions = []
        for text in X_test:
            prob_hateful = np.log(self.class_prior[1])
            prob_normal = np.log(self.class_prior[0])
            for word, count in text.items():
                if word in self.word_likelihoods:
                    prob_hateful += count * np.log(1.098876953125*self.word_likelihoods[word][0])
                    prob_normal += count * np.log(self.word_likelihoods[word][1])
                    print(prob_hateful)
                    print(prob_normal)
            predictions.append(1 if prob_hateful > prob_normal else 0)
        return predictions

# Example usage:
# print(prob_hateful)
# word_counts = ["love", "hate", "awesome", "stupid"]
# hateful_counts = [1, 3, 0, 1]
# normal_counts = [2, 0, 1, 2]
word_counts=[]
for word,count in words_captions_hateful.items():
    word_counts.append(word)
for word,count in words_captions_normal.items():
    if word not in word_counts:
        word_counts.append(word)


classifier = NaiveBayesClassifier()
classifier.train(word_counts,words_captions_hateful, words_captions_normal)

X_test = [{"black": 5}, {"bhai": 1, "everyone": 1}]
predictions = classifier.predict(X_test)
print(predictions)  # Output: [1, 0] indicating hateful and non-hateful respectively


-26.860697674372187
-32.57351554724379
-8.018123013969031
-7.985511585770869
[1, 0]


In [60]:
import numpy as np

class GaussianNaiveBayesClassifier:
    def __init__(self):
        self.class_prior = {}
        self.word_stats = {}

    def train(self, word_counts, hateful_dict, normal_dict):
        total_hateful=0
        total_normal=0
        for val in hateful_dict.values():
            total_hateful+=val
        for val in normal_dict.values():
            total_normal+=val
        self.class_prior[1] = probab_hateful
        self.class_prior[0] = 1 - probab_hateful
        
        # self.class_prior[1] = total_hateful / (total_hateful+total_normal)
        # self.class_prior[0] = total_normal / (total_hateful+total_normal)
        
        for word in word_counts:
            hateful_count=0
            normal_count=0
            if (word in hateful_dict):
                hateful_count=int(hateful_dict[word])
            if (word in normal_dict):
                normal_count=int(normal_dict[word])
            mean_hateful = hateful_count / total_hateful
            mean_normal = normal_count / total_normal
            variance_hateful = (hateful_count * (1 - mean_hateful)**2 + (total_hateful - hateful_count) * (0 - mean_hateful)**2) / total_hateful
            variance_normal = (normal_count * (1 - mean_normal)**2 + (total_normal - normal_count) * (0 - mean_normal)**2) / total_normal
            self.word_stats[word] = (mean_hateful, variance_hateful, mean_normal, variance_normal)

    def predict(self, X_test):
        predictions = []
        for text in X_test:
            prob_hateful = np.log(self.class_prior[1])
            prob_normal = np.log(self.class_prior[0])
            for word, count in text.items():
                if word in self.word_stats:
                    mean_hateful, variance_hateful, mean_normal, variance_normal = self.word_stats[word]
                    prob_hateful += self.gaussian_prob(count, mean_hateful, variance_hateful)
                    prob_normal += self.gaussian_prob(count, mean_normal, variance_normal)
            predictions.append(1 if prob_hateful > prob_normal else 0)
        return predictions


    def gaussian_prob(self, x, mean, variance):
        exponent = -((x - mean) ** 2) / (2 * (variance+1e-9))
        return np.exp(exponent) / np.sqrt(2 * np.pi * (variance+1e-9))

# Example usage:
# word_counts = ["love", "hate", "awesome", "stupid"]
# hateful_counts = [1, 3, 0, 1]
# normal_counts = [2, 0, 1, 2]
X_test=[]
answers=[]
with open ("../data/dev.jsonl") as f:
    for line in f: 
        elem=json.loads(line)
        temp_dict={}
        for word in elem['text'].split(" "):
            if word in temp_dict:
                temp_dict[word]+=1
            else:
                temp_dict[word]=1
        X_test.append(temp_dict)
        answers.append(elem['label'])

classifier = NaiveBayesClassifier()
classifier.train(word_counts, words_captions_hateful, words_captions_normal)

# X_test = [{"black": 100, "bitch": 100}, {"love": 1, "everyone": 1}]
# from decimal import Decimal

# low = Decimal('0.7')
# high = Decimal('1.5')
# precision = Decimal('1e-6')

# while high - low >= precision:
#     mid1 = low + (high - low) / Decimal('3')
#     mid2 = high - (high - low) / Decimal('3')
    
#     predictions1 = classifier.predict(X_test, float(mid1))
#     predictions2 = classifier.predict(X_test, float(mid2))
    
#     count1 = sum(1 for answer, predict in zip(answers, predictions1) if answer == predict)
#     count2 = sum(1 for answer, predict in zip(answers, predictions2) if answer == predict)
    
#     if count1 > count2:
#         high = mid2
#     else:
#         low = mid1

# best_factor = (low + high) / Decimal('2')

# print("Best factor:", float(best_factor))
# print("Max count:", max(count1, count2))



predictions = classifier.predict(X_test)
print(predictions)  # Output: [1, 0] indicating hateful and non-hateful respectively
count=0
count2=0
for answer,predict in zip(answers,predictions):
    if (answer==predict):
        if (answer==0):
            count+=1
        else:
            count2+=1
        # print("Haash")
        # count+=1
print(count,count2)


-6.049432836581898
-7.0455043272793985
-11.340335733341
-12.64408967162247
-15.929688038743777
-17.198198033674117
-21.171198180173302
-22.286693318830153
-24.95943113992401
-26.115198581254212
-33.153444820356704
-34.68052866227802
-42.81379556958282
-45.191768892357146
-11.452110218454179
-11.20438741063907
-17.32827133376204
-16.742348133165706
-24.653247167171124
-23.976443630252653
-35.41221020506536
-33.79453667977183
-10.32023203913526
-9.954561667916794
-18.999753535349654
-19.21303892950055
-26.92550322918767
-27.3263838867813
-35.101896663284386
-35.225389087904766
-56.61982273907285
-55.43693933184669
-60.93147991442587
-59.90911603601855
-66.28776557044782
-66.261473182738
-75.43729069590795
-75.85642268094297
-86.19625373380218
-86.3676629110221
-94.87577523001657
-95.26946522866712
-98.48717899672135
-98.96068909407211
-108.55299485405564
-108.55563859227708
-5.799621038185528
-5.766308101715874
-10.388973343588304
-10.320416463767522
-20.45478920092259
-19.91536596197249